# U.S. Medical Insurance Costs

# 1. Look over your dataset

Download a zip file here with the necessary datasets and an empty Jupyter Notebook where you can write your code.

Open insurance.csv and take a look at the file. Take note of how information is organized. How will this affect how you analyze the data in Python? Is there anything of particular interest to you in the dataset that you want to investigate? Think about these things before you jump into analyzing it.


* The data in the csv file is organized as a table: information of each person is stored in a row with 7 attributes in the columns:

    age, sex, bmi, children, smoker, region, charges
    
* Some of the columns are categorical (sex, smoker, region) while other are numerical (age: int, bmi: float, children: int, charges: float), all numbers are positive.
    
* There is no missing data

A library reading .csv files will be useful for importing the data. It would make sense to store the data in a dictionary with keys named according to the columns (variables) and values containing the list of all observations for each variable.

It would be intersting to know how much each of the 6 features - age, sex, BMI, number of children, whether the person smokes and region of residence - affects the charges (how much does the person pay for the medical insurance). But it is out of scope for now, we could go back to this question after we covered visualization and prediction (linear regression).


# 2. Scoping Your Project

Now that you have looked over your dataset, plan out what you want to analyze. What is it that you want to find out about this dataset? Based on the way information is organized, certain inspections may be easier to perform than others. As you map out the process, consider the scope of your analysis as well.

Properly scoping your project will greatly benefit you; scoping creates structure while requiring you to think through your entire project before you begin. You should start by stating the goals for your project, then gathering the data, and considering the analytical steps required. A proper project scope can be a great road map for your project, but keep in mind that some down-stream tasks may become dead ends which will require adjustment to the scope. 

We want to analyze the population itself and answer the following questions:
* Are people more likely to be healthy (no smoking, normal BMI) at younger age or older, with children or without them, if they are male or female?

    * Children vs. smoker - Are people more likely to be smokers with children or not?
    * Sex vs. smoker - Are females more likely to be smokers than males?
    * Age vs. smoker - which part of the population smokes the most? At what age do they usually start?
    * BMI vs. age - How does BMI distribution looks like for different age groups? Does BMI increase with age?
    * BMI vs. sex - How does BMI distribution looks like for males and for females? Which sex is more predisposed to obesity?
    * BMI vs. smoker - Do smokers have a higher BMI than non-smokers?
    * BMI vs. children - Does BMI increase with the number of children?
    
Since BMI and age variables are numerical, we should subdivide them into a few categories to simplify the analysis (since we decided to investigate the data without plotting).

# 3. Import your dataset

Import insurance.csv into your Python file and inspect the contents.


In [49]:
%ls

insurance.csv  LICENSE  README.md  us-medical-insurance-costs.ipynb


In [50]:
import csv
dataset = {}
with open("insurance.csv", newline='') as insurance_csv:
    dict_reader = csv.DictReader(insurance_csv) # converts the lines of our CSV file to Python dictionaries
    column_names = dict_reader.fieldnames
    for column_name in column_names:
        dataset[column_name] = []
    for row in dict_reader:
        # row is an ordered dictionary entry containing a tuple with the column heading and the data for each observation
        # key is the column heading and the value is data at each observation
        for key, value in row.items():
            dataset[key].append(value)
# test
for key, value in dataset.items():
    print("{}: {}".format(key, len(value)))

age: 1338
sex: 1338
bmi: 1338
children: 1338
smoker: 1338
region: 1338
charges: 1338


### Using Pandas
Borrowed from rj-pal

In [51]:
import pandas as pd

In [52]:
data = pd.read_csv('insurance.csv')

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [54]:
data.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


# 4. Save your dataset via Python variables

Save the features of your dataset (the columns) from insurance.csv by storing them in variables that can be used for analysis. As you consider what types of variables to use and how many you plan to create, think ahead about the parameters you wish to investigate and how your organization will impact this analysis.

# 5. Build out analysis functions or class methods

You now have everything you need to begin your analysis. You have organized the information from insurance.csv and have spent some time thinking about what it is you would like to investigate.

Now is the time to build out how you perform these investigations. Use the Python fundamentals you have learned so far to accomplish these tasks. There are many different ways you can achieve these analyses. In our hint, we will provide some ideas for how you can use Python to analyze data.

In [55]:
# borrowed from rj-pal
def variable_count(data, variable_key): # data is dataset, variable_key is the column you want to count
    count_dict = {}
    for i in range(len(data[variable_key])):
        if data[variable_key][i] not in count_dict:
            count_dict[data[variable_key][i]] = 1
        else:
            count_dict[data[variable_key][i]] += 1
    return count_dict

In [56]:
def variable_percentage(data, variable_key):
    counts = variable_count(data, variable_key)
    total = sum(counts.values())
    percentage_dict = {}
    for key, value in counts.items():
        percentage_dict[key] = round(value / total * 100, 2)
    return percentage_dict

In [57]:
def get_tuple_list(data, key1, key2):
    return list(zip(data[key1], data[key2]))

## BMI Analysis

### How many underweight, normal weight, overweight and obese people are in the dataset?

In [58]:
bmi_groups = { # key is the BMI group, value is the maximal value in the group
    "underweight": 18.5,
    "healthy weight": 25,
    "overweight": 30,
    "obese": float("inf")
}
def get_bmi_range(bmi):
    for key, value in bmi_groups.items():
        if bmi < value:
            return key
    return "obese"

In [59]:
# Add BMI range column to the dataset
data["bmi range"] = [get_bmi_range(bmi) for bmi in data["bmi"]]
data.head()

,age,sex,bmi,children,smoker,region,charges,bmi range
0,19,female,27.900,0,yes,southwest,16884.92400,overweight
1,18,male,33.770,1,no,southeast,1725.55230,obese
2,28,male,33.000,3,no,southeast,4449.46200,obese
3,33,male,22.705,0,no,northwest,21984.47061,healthy weight
4,32,male,28.880,0,no,northwest,3866.85520,overweight


In [60]:
bmi_range_count = variable_count(data, "bmi range")
bmi_range_count

{'overweight': 386, 'obese': 707, 'healthy weight': 225, 'underweight': 20}

In [61]:
# tesing variable_count using list method .count()
bmi_range_count = {}
bmi_ranges = data["bmi range"].values.tolist()
for bmi_group in bmi_groups.keys():
    bmi_range_count[bmi_group] = bmi_ranges.count(bmi_group)
bmi_range_count

{'underweight': 20, 'healthy weight': 225, 'overweight': 386, 'obese': 707}

In [81]:
# alternative using Counter from collections:
from collections import Counter
bmi_range_counts = Counter(bmi_ranges)
print(bmi_range_counts)

Counter({'obese': 707, 'overweight': 386, 'healthy weight': 225, 'underweight': 20})


In [62]:
bmi_range_percentage = variable_percentage(data, "bmi range")
bmi_range_percentage

{'overweight': 28.85,
 'obese': 52.84,
 'healthy weight': 16.82,
 'underweight': 1.49}

### BMI vs. age - What is BMI distribution for different age groups? Does it increase with age?

In [63]:
min(data.age)

18

In [64]:
max(data.age)

64

In [65]:
age_groups = [age for age in range(25, 66, 10)]
age_groups                                    

[25, 35, 45, 55, 65]

In [66]:
def values_by_age(data, age_groups, feature):
    age_group_to_values = {}
    # initialize the lists:
    for age_lim in age_groups:
        age_group_to_values["<" + str(age_lim)] = []
    # fill the lists:
    for age, value in get_tuple_list(data, "age", feature):
        for age_lim in age_groups:
            if age < age_lim:
                age_group_to_values["<" + str(age_lim)].append(value)
                break
    return age_group_to_values

In [67]:
def mean(l):
    return sum(l) / len(l)

In [74]:
# Find what BMI values fall into the 5 age groups:
age_group_to_bmi = values_by_age(data, age_groups, "bmi")
            
for age_group, bmi_list in age_group_to_bmi.items():
    print(f"For age group {age_group}, BMI ranges between {round(min(bmi_list), 2)} and {round(max(bmi_list), 2)}, mean BMI is {round(mean(bmi_list), 2)}")

For age group <25, BMI ranges between 15.96 and 53.13, mean BMI is 30.04
For age group <35, BMI ranges between 17.2 and 46.53, mean BMI is 30.06
For age group <45, BMI ranges between 16.82 and 47.6, mean BMI is 30.4
For age group <55, BMI ranges between 18.05 and 48.07, mean BMI is 31.15
For age group <65, BMI ranges between 18.34 and 49.06, mean BMI is 31.76


### BMI vs. sex - What is BMI distribution for males and for females? Which sex is more predisposed to obesity?

In [89]:
def values_by_feature(data, feature1, feature2):
    feature1_by_feature2 = {}
    for feature1, feature2 in get_tuple_list(data, feature1, feature2):
        if feature2 not in feature1_by_feature2:
            feature1_by_feature2[feature2] = [feature1]
        else:
            feature1_by_feature2[feature2].append(feature1)
    return feature1_by_feature2

In [90]:
# Subdivide the BMI by sex:
bmi_by_sex = values_by_feature(data, "bmi", "sex")
for sex, bmi_list in bmi_by_sex.items():
    print(f"For {sex}s, BMI ranges between {round(min(bmi_list), 2)} and {round(max(bmi_list), 2)}, mean BMI is {round(mean(bmi_list), 2)}")

For females, BMI ranges between 16.82 and 48.07, mean BMI is 30.38
For males, BMI ranges between 15.96 and 53.13, mean BMI is 30.94


### BMI vs. smoker - Do smokers have a higher BMI than non-smokers?

In [91]:
# Subdivide the BMI by smoker:
bmi_by_smoker = values_by_feature(data, "bmi", "smoker")
for smoker, bmi_list in bmi_by_smoker.items():
    if smoker == "yes":
        print(f"For smokers, BMI ranges between {round(min(bmi_list), 2)} and {round(max(bmi_list), 2)}, mean BMI is {round(mean(bmi_list), 2)}")
    else: 
        print(f"For non-smokers, BMI ranges between {round(min(bmi_list), 2)} and {round(max(bmi_list), 2)}, mean BMI is {round(mean(bmi_list), 2)}")

For smokers, BMI ranges between 17.2 and 52.58, mean BMI is 30.71
For non-smokers, BMI ranges between 15.96 and 53.13, mean BMI is 30.65


### Conclusions
* According to this dataset, over a half of US adult popolation is obese, over a quarter are overweight, only ~17% are in a healthy weight range and ~1.5% are underweight, according to the commonly used BMI ranges (https://www.nhs.uk/common-health-questions/lifestyle/what-is-the-body-mass-index-bmi/, https://www.cdc.gov/healthyweight/assessing/bmi/adult_bmi/index.html).
* As expected, mean BMI increases with age, but not substantially. It is interesting that younger people (< 25 years) despite having the lowest mean BMI (30.04) tend to be more underweight (minimal BMI 15.96) or obese (maximal BMI 53.13), that is the spread is higher.
* Male population has a higher BMI spread than female population (15.96 - 53.13 vs. 16.82 - 48.07) with a slightly higher mean (30.94 vs. 30.38).
* Smokers do not have a substantially higher mean BMI than non-smokers (30.71 vs. 30.65).

# 6. Project Extensions

You’re welcome to expand your analysis beyond what you have already done! Some potential extra features to add to your portfolio project are the following:

* Organize your findings into dictionaries, lists, or another convenient datatype.
* Make predictions about what features are the most influential for an individual’s medical insurance charges based on your analysis.
* Explore areas where the data may include bias and how that would impact potential use cases.

Congrats on completing your portfolio project!
